In [89]:
'''imports'''

'''basic imports'''
import numpy as np
import matplotlib.pyplot as plt
'''import geopandas as gp'''
import scipy
import sparse
from scipy.sparse import csr_matrix
import executor as exc

'''dataframe handling'''
'''import pandas as pd'''
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import dask_ml.cluster
import dask_ml.preprocessing
from dask_ml.preprocessing import OneHotEncoder
import joblib

import datetime as date

'''learning'''
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [90]:
'''Lists'''

'''Genera, species library'''
aedes = ['dorsalis', 'melanimon', 'nigromaculis', 'sierrensis', 'squamiger', 'vexans', 'washinoi']
anopheles = ['franciscanus', 'freeborni', 'occidentalis', 'punctipennis']
culex = ['apicalis', 'erythrothorax', 'pipiens', 'restuans', 'stigmatosoma', 'tarsalis']
culiseta = ['incidens', 'inornata', 'particeps']
genera = [aedes, anopheles, culex, culiseta]

'''CSV / dataframe column names'''
allcols=['agency_collection_num','collection_id','code','longitude','latitude','collection_date','aedes_dorsalis_males',
         'aedes_dorsalis_females-mixed','aedes_melanimon_males','aedes_melanimon_females-mixed','aedes_nigromaculis_males',
         'aedes_nigromaculis_females-mixed','aedes_sierrensis_males','aedes_sierrensis_females-mixed',
         'aedes_squamiger_males','aedes_squamiger_females-mixed','aedes_vexans_males','aedes_vexans_females-mixed',
         'aedes_washinoi_males','aedes_washinoi_females-mixed','anopheles_franciscanus_males',
         'anopheles_franciscanus_females-mixed','anopheles_freeborni_males','anopheles_freeborni_females-mixed',
         'anopheles_occidentalis_males','anopheles_occidentalis_females-mixed','anopheles_punctipennis_males', 
         'anopheles_punctipennis_females-mixed','culex_apicalis_males','culex_apicalis_females-mixed',
         'culex_erythrothorax_males','culex_erythrothorax_females-mixed','culex_pipiens_males',
         'culex_pipiens_females-mixed','culex_restuans_males','culex_restuans_females-mixed','culex_stigmatosoma_males',
         'culex_stigmatosoma_females-mixed','culex_tarsalis_males','culex_tarsalis_females-mixed','culiseta_incidens_males',
         'culiseta_incidens_females-mixed','culiseta_inornata_males','culiseta_inornata_females-mixed',
         'culiseta_particeps_males','culiseta_particeps_females-mixed']

rawabundcols = ['aedes_dorsalis_males','aedes_dorsalis_females-mixed','aedes_melanimon_males','aedes_melanimon_females-mixed',
             'aedes_nigromaculis_males','aedes_nigromaculis_females-mixed','aedes_sierrensis_males',
             'aedes_sierrensis_females-mixed','aedes_squamiger_males','aedes_squamiger_females-mixed','aedes_vexans_males',
             'aedes_vexans_females-mixed','aedes_washinoi_males','aedes_washinoi_females-mixed','anopheles_franciscanus_males',
             'anopheles_franciscanus_females-mixed','anopheles_freeborni_males','anopheles_freeborni_females-mixed',
             'anopheles_occidentalis_males','anopheles_occidentalis_females-mixed','anopheles_punctipennis_males',
             'anopheles_punctipennis_females-mixed','culex_apicalis_males','culex_apicalis_females-mixed',
             'culex_erythrothorax_males','culex_erythrothorax_females-mixed','culex_pipiens_males',
             'culex_pipiens_females-mixed','culex_restuans_males','culex_restuans_females-mixed','culex_stigmatosoma_males',
             'culex_stigmatosoma_females-mixed','culex_tarsalis_males','culex_tarsalis_females-mixed','culiseta_incidens_males',
             'culiseta_incidens_females-mixed','culiseta_inornata_males','culiseta_inornata_females-mixed',
             'culiseta_particeps_males','culiseta_particeps_females-mixed']

'''column names of findata'''
fincols = ['agency_collection_num','collection_id','code','longitude','latitude','collection_date','dorsalis','melanimon',
           'nigromaculis','sierrensis','squamiger','vexans','washinoi','franciscanus','freeborni','occidentalis','punctipennis',
           'apicalis','erythrothorax','pipiens','restuans','stigmatosoma','tarsalis','incidens','inornata','particeps']

In [103]:
'''data management'''
'''findata is the dataframe containing all processed abundance data with supplemental info needed later'''
'''testdata is the processed abundance dataframe to be used in the clustering'''


supportcols = ['agency_collection_num','collection_id','code','longitude','latitude','collection_date']

findata = dd.read_csv (r'C:\Users\andre\abundance1622.csv', usecols = supportcols, parse_dates=['collection_date'])


proabundcols = ['dorsalis', 'melanimon', 'nigromaculis', 'sierrensis', 'squamiger','vexans', 'washinoi', 'franciscanus', 
                'freeborni', 'occidentalis', 'punctipennis', 'apicalis', 'erythrothorax','pipiens','restuans', 'stigmatosoma', 
                'tarsalis', 'incidens', 'inornata', 'particeps']


abunddata = dd.read_csv (r'C:\Users\andre\abundance1622.csv', usecols = rawabundcols)
testdata = dd.read_csv (r'C:\Users\andre\abundance1622.csv', usecols = ['collection_id'])

for x in proabundcols: 
    sera = abunddata.iloc[:,2*proabundcols.index(x)]
    serb = abunddata.iloc[:,(2*proabundcols.index(x))+1]
    serret = sera.add(serb, fill_value = 0)
    findata[x] = serret 
    testdata[x] = serret

testdata.set_index('collection_id', sorted = True, npartitions = 'auto')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 34828: invalid start byte

In [104]:
'''Time for the machines to learn'''

testdata
ohen = OneHotEncoder(categories = proabundcols)
kmeans = dask_ml.cluster.KMeans(n_clusters=5)
X = ohen.transform(testdata)
kmeans.fit(X,njobs=-1)

AttributeError: 'OneHotEncoder' object has no attribute 'categories_'